# Random Forests 3

---

__This Notebook__:

- performs 3 grid searches:

    10. X_bot_tfidf_svd_cos
    11. X_bot_tfidf_svd_feat
    12. X_bot_tfidf_svd_feat_cos
  
  
__Command Line__:

I also ran the `X_bot` again since I accidentally overwrote that grid search:

```
$python run_gridsearches.py
Training 1: X_bot...
Elapsed: 1630 s
Best params: {'max_depth': 20, 'max_features': 500, 'min_samples_split': 5, 'n_e
stimators': 100}
Confusion matrix on validation set:
     pred_neg  pred_pos
neg       843         3
pos        15       114
Evaluation metrics:
accuracy: 0.9815
sensitivity: 0.8837
specificity: 0.9965

Tot elapsed: 27 m 10 s
```

It took long because I used `n_jobs=4`. This got saved as `01062021_rf_gridsearches_3.joblib` in the `data/3_modeling/` folder.

## Setup

In [1]:
import os
import re
import time
import joblib

import numpy as np
import pandas as pd
import scipy.sparse as sp

from datetime import datetime
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, recall_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-01-12


## Load Data

In [2]:
# load target
raw_path = os.path.join("data","1_raw")
filename = "y_train.csv"
y = pd.read_csv(os.path.join(raw_path, filename))
y = np.array(y.iloc[:,0].ravel())
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load 12 matrices
proc_dir = os.path.join("data","2_processed")
Xnames = [x for x in os.listdir(proc_dir) if re.search('.npz', x)]
Xs = []
for ix, X in enumerate(Xnames):
    path_ = os.path.join(proc_dir, Xnames[ix])
    Xs.append(sp.load_npz(path_))

In [3]:
# 12 representations
for ix, Xname in enumerate(Xnames):
    Xname = Xname.split('.')[0]
    print(ix+1, Xname)

1 X_bot
2 X_bot_feat
3 X_bot_svd
4 X_bot_svd_cos
5 X_bot_svd_feat
6 X_bot_svd_feat_cos
7 X_bot_tfidf
8 X_bot_tfidf_feat
9 X_bot_tfidf_svd
10 X_bot_tfidf_svd_cos
11 X_bot_tfidf_svd_feat
12 X_bot_tfidf_svd_feat_cos


## Grid search random forest models

Using sklearn's GridSearchCV with 10-fold cross validation on a shallow param grid, varying representations.

In [4]:
def print_eval_metrics(y_val, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    acc = (tp + tn) / (tp + tn + fp + fn)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    print(f'accuracy: {acc:0.4f}')
    print(f'sensitivity: {tpr:0.4f}')
    print(f'specificity: {tnr:0.4f}')

In [5]:
def gridsearch_wrapper(Xs, Xnames, test=False, k=10):
    """
    Performs grid searches and collects them in a list.
    Args:
        Xs: the numeric matrices
        Xnames: their names
        test: faster, shallower searches for testing
        k: the number of CV folds
    """
    start_time = time.time()
    model_dir = os.path.join("data", "3_modeling")
    
    # instantiate list of dicts to gather results
    gridsearches = []
    for ix, X_name in enumerate(Xnames):

        X_ = Xs[ix].toarray()
        X_name = X_name.split('.')[0]

        # split into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X_, y, stratify=y)

        # setup testing param grid
        test_param_grid = {
            'min_samples_split': [5, 10, 20], 
            'n_estimators' : [50, 100],
            'max_depth': [10, 20],
            'max_features': [50, 100, 200]
        }

        # setup param grid for final not-too-deep search
        param_grid = {
            'min_samples_split': [5, 10, 15],
            'n_estimators' : [100, 200],
            'max_depth': [5, 10, 20],
            'max_features': [50, 100, 250, 500]
        }

        # setup scorers
        scorers = {
            'acc': make_scorer(accuracy_score),
            'tpr': make_scorer(recall_score, pos_label=1), # sensitivity, recall
            'tnr': make_scorer(recall_score, pos_label=0) # specificity, selectivity
        }

        # instantiate estimator
        clf = RandomForestClassifier(n_jobs=-1, random_state=42)

        # instantiate k-fold gridsearch
        cv_folds = StratifiedKFold(n_splits=k)
    
        if test == True:
            grid_search_clf = GridSearchCV(clf, test_param_grid, # test grid
                                           scoring=scorers, 
                                           refit='tpr', cv=cv_folds, 
                                           return_train_score=True, n_jobs=-1)
        else:
            grid_search_clf = GridSearchCV(clf, param_grid,
                                           scoring=scorers, 
                                           refit='tpr', cv=cv_folds, 
                                           return_train_score=True, n_jobs=-1)           

        # train models
        print(f'\nTraining {ix+1}: {X_name}...')
        start_gs = time.time()
        grid_search_clf.fit(X_train, y_train)
        elapsed_secs = time.time() - start_gs
        print(f'Elapsed: {elapsed_secs:0.0f} s')

        # predict
        y_pred = grid_search_clf.predict(X_val)
        print(f'Best params: {grid_search_clf.best_params_}')

        # confusion matrix on validation set
        print(f'Confusion matrix on validation set:')
        print(pd.DataFrame(confusion_matrix(y_val, y_pred),
                           columns=['pred_neg', 'pred_pos'],
                           index=['neg', 'pos']))
        # eval metrics
        print('Evaluation metrics:')
        print_eval_metrics(y_val, y_pred)

        data = {'representation':X_name,
                'gridsearch_res':grid_search_clf}
        
        # save gridsearch
        filename = ''.join([str(ix+1), "_", X_name, "_rf_gridsearch.joblib"])
        file_path = os.path.join(model_dir, filename)                                                    
        joblib.dump(data, file_path)
        
        # gather results into a list of dicts
        gridsearches.append(data)
        
    mins, secs = divmod(time.time() - start_time, 60)
    print(f'\nTot elapsed: {mins:0.0f} m {secs:0.0f} s')
    return gridsearches

In [6]:
Xnames[9:]

['X_bot_tfidf_svd_cos.npz',
 'X_bot_tfidf_svd_feat.npz',
 'X_bot_tfidf_svd_feat_cos.npz']

In [7]:
results = gridsearch_wrapper(Xs=Xs[9:], Xnames=Xnames[9:], test=False, k=10)


Training 1: X_bot_tfidf_svd_cos...
Elapsed: 5297 s
Best params: {'max_depth': 5, 'max_features': 250, 'min_samples_split': 5, 'n_estimators': 200}
Confusion matrix on validation set:
     pred_neg  pred_pos
neg       838         8
pos        18       111
Evaluation metrics:
accuracy: 0.9733
sensitivity: 0.8605
specificity: 0.9905

Training 2: X_bot_tfidf_svd_feat...
Elapsed: 3315 s
Best params: {'max_depth': 20, 'max_features': 250, 'min_samples_split': 15, 'n_estimators': 100}
Confusion matrix on validation set:
     pred_neg  pred_pos
neg       845         1
pos        15       114
Evaluation metrics:
accuracy: 0.9836
sensitivity: 0.8837
specificity: 0.9988

Training 3: X_bot_tfidf_svd_feat_cos...
Elapsed: 3473 s
Best params: {'max_depth': 10, 'max_features': 100, 'min_samples_split': 5, 'n_estimators': 200}
Confusion matrix on validation set:
     pred_neg  pred_pos
neg       842         4
pos        16       113
Evaluation metrics:
accuracy: 0.9795
sensitivity: 0.8760
specificity:

## Persist results

In [8]:
results

[{'representation': 'X_bot_tfidf_svd_cos',
  'gridsearch_res': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
               estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
               n_jobs=-1,
               param_grid={'max_depth': [5, 10, 20],
                           'max_features': [50, 100, 250, 500],
                           'min_samples_split': [5, 10, 15],
                           'n_estimators': [100, 200]},
               refit='tpr', return_train_score=True,
               scoring={'acc': make_scorer(accuracy_score),
                        'tnr': make_scorer(recall_score, pos_label=0),
                        'tpr': make_scorer(recall_score, pos_label=1)})},
 {'representation': 'X_bot_tfidf_svd_feat',
  'gridsearch_res': GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
               estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
               n_jobs=-1,
               

In [9]:
model_dir = os.path.join("data", "3_modeling")
file_path = os.path.join(model_dir, "".join(["01062021", "_rf_gridsearches_2`.joblib"]))

# save all gridsearches
joblib.dump(results, file_path)

['data\\3_modeling\\01062021_rf_gridsearches_2`.joblib']

---